**9.	Which player has won the most tournaments in a single month?**


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

Reading requierd tables

In [2]:
MatchTournamentInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchTournamentInfo.csv')
MatchEventInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchEventInfo.csv')
MatchHomeTeamInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchHomeTeamInfo.csv')
MatchAwayTeamInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchAwayTeamInfo.csv')

Drop duplicated data

Creating a new table which contains match id, winer name and id and tournament name

In [3]:
merged_HW = MatchHomeTeamInfo[['match_id' ,
                                  'player_id']].merge(MatchAwayTeamInfo[['match_id' ,
                                                                         'player_id']],
                                                                         on ='match_id').drop_duplicates(subset='match_id')
merged_EHW = merged_HW[['match_id' ,
                        'player_id_x' , 'player_id_y']].merge(MatchEventInfo[['match_id',
                                                                              'winner_code',
                                                                              'start_datetime']],
                                                                               on ='match_id')
merged_TEHW = merged_EHW[['match_id' ,
                        'player_id_x',
                        'player_id_y',
                        'winner_code',
                        'start_datetime']].merge(MatchTournamentInfo[['match_id',
                                                                      'tournament_name',
                                                                      'tournament_id']],
                                                                       on ='match_id')
merged_TEHW = merged_TEHW.dropna()
merged_TEHW['winner_id'] = merged_TEHW.apply( lambda x : x['player_id_x']  
                                             if x['winner_code'] == 1 
                                             else x['player_id_y'] , axis = 1)


In [4]:
merged_TEHW.head()

,match_id,player_id_x,player_id_y,winner_code,start_datetime,tournament_name,tournament_id,winner_id
0,12260075,14486,254742,2.0,1714499700,"Madrid, Spain",129164,254742
1,12260076,206570,90080,1.0,1714480500,"Madrid, Spain",129164,206570
2,12260077,46391,275923,2.0,1714484700,"Madrid, Spain",129164,275923
3,12260078,106755,275923,1.0,1714564800,"Madrid, Spain",129164,106755
4,12260078,106755,275923,1.0,1714564800,"Madrid, Spain",129164,106755


determinig the last match and its winner according to start datetime

In [11]:
player_tounament = merged_TEHW.groupby('tournament_id',
                                        as_index=False).agg({'start_datetime': 'max' ,
                                                             'winner_id' :'last'})
player_tounament.head()

,tournament_id,start_datetime,winner_id
0,129164,1714929300,106755
1,129201,1714842900,228272
2,129510,1714891500,275471
3,129514,1714923000,48225
4,129518,1714922100,102151


calculating number of won tournoments for each player

In [21]:
players = player_tounament.groupby('winner_id', as_index= False).aggregate(no_tour = ('tournament_id' , 'count'))
players.head(2)

,winner_id,no_tour
0,15983,2
1,18919,1


**Reporting the best player with the most wins**

In [23]:
best_no_tour = players['no_tour'].max()
best_winer_id = players['winner_id'].loc[players['no_tour'] == players['no_tour'].max()].values[0]
print (f'Player  with {best_winer_id} ID has won {best_no_tour} tournomernt which is most tournaments in a single month')


Player  with 228272 ID has won 3 tournomernt which is most tournaments in a single month
